# Tiny FSDL Walkthrough

In [ ]:
from tiny_fsdl import Preprocess 


input_dir = "./ml/data/raw_images/data1"
output_dir = "./ml/data/processed_images/data1"

# Relative locations
y = 11
w = 15
h = 25
digits_before = 4
digits_after = 2
x_before = 46
x_after = 89
    
preprocess = Preprocess(y, w, h)
preprocess.start(
    input_dir=input_dir,
    output_dir=output_dir,
    digits_before=digits_before,
    digits_after=digits_after,
    x_before=x_before,
    x_after=x_after,
)

## Train

In [ ]:
from tiny_fsdl.train import prep_data
from tiny_fsdl.train import Trainer


# General Params
seed = 1337

# Data Params
label_dir = "../ml/data/raw_images/labeled_bmp_data/"
split_dir = "../ml/data/raw_images/split_data/"
split_ratio = (0.7, 0.2, 0.1)

# Training Params
model_output_dir = "../ml/models/"
model_filename = "digit_model.h5"
num_classes = 10
batch_size = 32
img_size = (15, 25)
base_learning_rate = 0.001
epochs = 100

train_dataset, validation_dataset, test_dataset = prep_data(
    label_dir, 
    split_dir, 
    split_ratio, 
    batch_size, 
    img_size, 
    seed
)
trainer = Trainer(train_dataset, validation_dataset, test_dataset)
trainer.build_model(img_size, num_classes)
trainer.evaluate(validation=True, test=False)
trainer.train(epochs=epochs)
trainer.evaluate(validation=True, test=True)
trainer.save_model(output_dir=model_output_dir, filename="digit_model.h5")

## Convert to TFLite and Quantize

In [5]:
from tiny_fsdl.quantize import quantize_model, representative_dataset, test_quantized_model

quantized_path = "../ml/models/digit_model_quantized.tflite"

orig_model_path = os.path.join(model_output_dir, "digit_model.h5")
representative_dataset = representative_dataset()
tflite_model = quantize_model(model_ou, representative_dataset)

test_quantized_model()